In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


In [2]:
base_dir = 'tomato_new'
img_height, img_width = 224, 224  
batch_size = 32  

In [3]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [4]:
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

Found 8800 images belonging to 10 classes.


In [5]:
validation_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 2200 images belonging to 10 classes.


In [6]:
def create_model():
    model = Sequential([
        # 1st Convolutional Block
        Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
        BatchNormalization(),
        
        # 2nd Convolutional Block
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),

        # 3rd Convolutional Block
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        
        # 4th Convolutional Block
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        
        # 5th Convolutional Block
        Conv2D(256, (3, 3), activation='relu'),
        BatchNormalization(),
        
        # 6th Convolutional Block
        Conv2D(256, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),

        # 7th Convolutional Block
        Conv2D(512, (3, 3), activation='relu'),
        BatchNormalization(),

        # 8th Convolutional Block
        Conv2D(512, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),

        # Flatten Layer
        Flatten(),

        # Dense Layers
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')  # Update to 10 classes for tomatoes
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [7]:
model = create_model()

In [8]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model_tomato23.kears', save_best_only=True, monitor='val_loss')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

In [ ]:
history = model.fit(
    train_generator,
    epochs=75,  # Adjust based on model convergence
    validation_data=validation_generator,
    callbacks=[early_stopping, model_checkpoint, reduce_lr]
)

Epoch 1/75
205/275 [=====================>........] - ETA: 4:30 - loss: 8.5890 - accuracy: 0.2674